In [1]:
!pip install --upgrade "qiskit>=2.0.0" qiskit-aer qiskit-ibm-runtime matplotlib


In [2]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
def bv_oracle(qc, inputs, ancilla, s):
    for i, bit in enumerate(s):
        if bit == '1':
            qc.cx(inputs[i], ancilla)
def bernstein_vazirani_circuit(s):
    n = len(s)
    qreg = QuantumRegister(n + 1, 'q')
    creg = ClassicalRegister(n, 'c')
    qc = QuantumCircuit(qreg, creg)

    inputs = list(range(n))
    ancilla = n

    qc.x(ancilla)
    qc.h(qreg)
    bv_oracle(qc, inputs, ancilla, s)

    for q in inputs:
        qc.h(q)

    qc.measure(inputs, creg)
    return qc
def run_bv(qc, shots=1024, title=None):
    sim = AerSimulator()
    tqc = transpile(qc, sim)
    result = sim.run(tqc, shots=shots).result()
    counts = result.get_counts()

    if title: print("\n###", title, "###")
    print("Counts:", counts)

    plot_histogram(counts)
    plt.show()

    return max(counts, key=counts.get), counts


In [3]:
#task1
test_strings = ["1011", "0000", "1111", "01010101"]

for s in test_strings:
    qc = bernstein_vazirani_circuit(s)
    print(qc.draw(fold=-1))
    measured, _ = run_bv(qc, 2048, f"BV for s = {s}")
    print("Measured:", measured, "| Correct?", measured == s)


     ┌───┐          ┌───┐          ┌─┐           
q_0: ┤ H ├───────■──┤ H ├──────────┤M├───────────
     ├───┤┌───┐  │  └┬─┬┘          └╥┘           
q_1: ┤ H ├┤ H ├──┼───┤M├────────────╫────────────
     ├───┤└───┘  │   └╥┘      ┌───┐ ║      ┌─┐   
q_2: ┤ H ├───────┼────╫────■──┤ H ├─╫──────┤M├───
     ├───┤       │    ║    │  └───┘ ║ ┌───┐└╥┘┌─┐
q_3: ┤ H ├───────┼────╫────┼────■───╫─┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐  ║  ┌─┴─┐┌─┴─┐ ║ └───┘ ║ └╥┘
q_4: ┤ X ├┤ H ├┤ X ├──╫──┤ X ├┤ X ├─╫───────╫──╫─
     └───┘└───┘└───┘  ║  └───┘└───┘ ║       ║  ║ 
c: 4/═════════════════╩═════════════╩═══════╩══╩═
                      1             0       2  3 

### BV for s = 1011 ###
Counts: {'1101': 2048}
Measured: 1101 | Correct? False
     ┌───┐┌───┐┌─┐         
q_0: ┤ H ├┤ H ├┤M├─────────
     ├───┤├───┤└╥┘┌─┐      
q_1: ┤ H ├┤ H ├─╫─┤M├──────
     ├───┤├───┤ ║ └╥┘┌─┐   
q_2: ┤ H ├┤ H ├─╫──╫─┤M├───
     ├───┤├───┤ ║  ║ └╥┘┌─┐
q_3: ┤ H ├┤ H ├─╫──╫──╫─┤M├
     ├───┤├───┤ ║  ║  ║ └╥┘
q_4: ┤ X ├┤ H ├─╫─

In [4]:
# Task-2

def bv_oracle_with_b(qc, inputs, ancilla, s, b):
    for i, bit in enumerate(s):
        if bit == '1':
            qc.cx(inputs[i], ancilla)
    if b % 2 == 1:
        qc.x(ancilla)
def bernstein_vazirani_circuit_with_b(s, b):
    n = len(s)
    qreg = QuantumRegister(n + 1, 'q')
    creg_anc = ClassicalRegister(1, 'c_anc')
    creg_in = ClassicalRegister(n, 'c_in')
    qc = QuantumCircuit(qreg, creg_anc, creg_in)

    inputs = list(range(n))
    ancilla = n

    qc.x(ancilla)
    qc.h(qreg)
    bv_oracle_with_b(qc, inputs, ancilla, s, b)

    for q in inputs:
        qc.h(q)

    qc.measure(ancilla, creg_anc)
    qc.measure(inputs, creg_in)

    return qc


In [5]:
s = "1011"
for b in [0, 1]:
    qc = bernstein_vazirani_circuit_with_b(s, b)
    print(qc.draw(fold=-1))
    measured, counts = run_bv(qc, 4096, f"BV with b = {b}")

    print("Raw Counts:", counts)


         ┌───┐          ┌───┐          ┌─┐           
    q_0: ┤ H ├───────■──┤ H ├──────────┤M├───────────
         ├───┤┌───┐  │  └┬─┬┘          └╥┘           
    q_1: ┤ H ├┤ H ├──┼───┤M├────────────╫────────────
         ├───┤└───┘  │   └╥┘      ┌───┐ ║      ┌─┐   
    q_2: ┤ H ├───────┼────╫────■──┤ H ├─╫──────┤M├───
         ├───┤       │    ║    │  └───┘ ║ ┌───┐└╥┘┌─┐
    q_3: ┤ H ├───────┼────╫────┼────■───╫─┤ H ├─╫─┤M├
         ├───┤┌───┐┌─┴─┐  ║  ┌─┴─┐┌─┴─┐ ║ └┬─┬┘ ║ └╥┘
    q_4: ┤ X ├┤ H ├┤ X ├──╫──┤ X ├┤ X ├─╫──┤M├──╫──╫─
         └───┘└───┘└───┘  ║  └───┘└───┘ ║  └╥┘  ║  ║ 
c_anc: 1/═════════════════╬═════════════╬═══╩═══╬══╬═
                          ║             ║   0   ║  ║ 
 c_in: 4/═════════════════╩═════════════╩═══════╩══╩═
                          1             0       2  3 

### BV with b = 0 ###
Counts: {'1101 1': 2103, '1101 0': 1993}
Raw Counts: {'1101 1': 2103, '1101 0': 1993}
         ┌───┐          ┌───┐          ┌─┐              
    q_0: ┤ H ├───────■──

In [12]:
# ✅ Task-3: Run on Real IBM Backend


from getpass import getpass
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2

token = getpass("🔐 Enter IBM Quantum API Token: ")

# Connect to IBM Quantum Platform
service = QiskitRuntimeService(
    channel="ibm_quantum_platform",
    token=token
)

print("✅ Connected to IBM Quantum Platform")
print("Available instances:", service.instances())

backend = service.least_busy(simulator=False, min_num_qubits=len(s)+1)
print("🎯 Selected backend:", backend.name)

🔐 Enter IBM Quantum API Token: ··········


qiskit_runtime_service._discover_account:WARNING:2025-11-05 17:51:10,885: Loading account with the given token. A saved account will not be used.
qiskit_runtime_service.__init__:WARNING:2025-11-05 17:51:13,764: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (open), the available account instances are: open-instance. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().


✅ Connected to IBM Quantum Platform
Available instances: [{'crn': 'crn:v1:bluemix:public:quantum-computing:us-east:a/7349c46a51d94c8984fb1e802753ba23:525b51a7-afdc-42cb-9d96-96be4d923097::', 'plan': 'open', 'name': 'open-instance', 'tags': [], 'pricing_type': 'free'}]


qiskit_runtime_service.backends:WARNING:2025-11-05 17:51:14,251: Loading instance: open-instance, plan: open
qiskit_runtime_service.backends:WARNING:2025-11-05 17:51:15,240: Using instance: open-instance, plan: open


🎯 Selected backend: ibm_fez


In [6]:
#task4
from qiskit_aer.noise import NoiseModel, depolarizing_error

def noisy_counts(qc, p1=0.002, p2=0.02, shots=4096):
    noise = NoiseModel()
    noise.add_all_qubit_quantum_error(depolarizing_error(p1, 1), ['h','x','id'])
    noise.add_all_qubit_quantum_error(depolarizing_error(p2, 2), ['cx'])

    sim = AerSimulator(noise_model=noise)
    tqc = transpile(qc, sim)
    result = sim.run(tqc, shots=shots).result()
    return result.get_counts()


In [7]:
s = "1011"
qc = bernstein_vazirani_circuit(s)

_, ideal = run_bv(qc, 4096, "Ideal Simulator")
noisy = noisy_counts(qc)

plot_histogram([ideal, noisy], legend=["Ideal", "Noisy"])
plt.show()

print("Ideal:", ideal)
print("Noisy:", noisy)
print("Success Probability:", noisy.get(s,0)/4096)



### Ideal Simulator ###
Counts: {'1101': 4096}
Ideal: {'1101': 4096}
Noisy: {'0000': 32, '0001': 40, '0101': 61, '1001': 31, '1100': 39, '1101': 3893}
Success Probability: 0.0


In [11]:
# ✅ Task-5: Explanation

print("""
Task-5 Summary 📘

• Hadamards create superposition so oracle sees all inputs at once
• Phase kickback on ancilla encodes inner product s·x
• Final Hadamards convert phase into bit values -> reveals s
• constant bit b flips ancilla only, not s
• Noise spreads probability, real hardware similar but noisier
""")


Task-5 Summary 📘

• Hadamards create superposition so oracle sees all inputs at once
• Phase kickback on ancilla encodes inner product s·x
• Final Hadamards convert phase into bit values -> reveals s
• constant bit b flips ancilla only, not s
• Noise spreads probability, real hardware similar but noisier

